In [37]:
import pandas as pd
import os

RAW_PATH = '../data/raw/' 
PROCESSED_PATH = '../data/processed/'

os.makedirs(PROCESSED_PATH, exist_ok=True)
print("✅ Paths are set and ready.")

✅ Paths are set and ready.


We load the datasets and fix the encoding for the translation file to avoid corrupted characters.

In [38]:
products = pd.read_csv(f'{RAW_PATH}olist_products_dataset.csv')
sellers = pd.read_csv(f'{RAW_PATH}olist_sellers_dataset.csv')
translation = pd.read_csv(f'{RAW_PATH}product_category_name_translation.csv', encoding='utf-8-sig')

products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


We drop column product_name_lenght which provides no value for business or logistics analysis.

In [39]:
products = products.drop(columns=['product_name_lenght'])
print("🗑️ Useless columns removed.")

🗑️ Useless columns removed.


We handle nulls by filling them with "unknown" and standardize category names to lowercase for consistent grouping in SQL.

In [40]:
products['product_category_name'] = products['product_category_name'].fillna('unknown')
products['product_category_name'] = products['product_category_name'].str.lower().str.replace(' ', '_')

products = products.merge(translation, on='product_category_name', how='left')
products.head()

,product_id,product_category_name,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,402.0,4.0,625.0,20.0,17.0,13.0,housewares


We address data quality by filling 610 missing values in product_category_name with the label "unknown". We also transform all category names to lowercase and replace spaces with underscores. This ensures that SQL grouping operations are accurate and not split by casing or formatting differences.

In [41]:
# 1. Filling missing category names
products['product_category_name'] = products['product_category_name'].fillna('unknown')

# 2. String normalization (lowercase and underscores)
products['product_category_name'] = products['product_category_name'].str.lower().str.replace(' ', '_')

# 3. Quality Check: Confirm no nulls remain in categories
print(f"Missing values after cleaning: {products['product_category_name'].isnull().sum()}")
products['product_category_name'].value_counts().head(10)

Missing values after cleaning: 0


product_category_name
cama_mesa_banho           3029
esporte_lazer             2867
moveis_decoracao          2657
beleza_saude              2444
utilidades_domesticas     2335
automotivo                1900
informatica_acessorios    1639
brinquedos                1411
relogios_presentes        1329
telefonia                 1134
Name: count, dtype: int64

We merge the products table with the English translation dataset. To ensure 100% data quality, we manually map specific categories like pc_gamer and unknown that are missing from the original translation file.

In [42]:
translation.columns = translation.columns.str.strip()

products = products.merge(translation, on='product_category_name', how='left')

manual_map = {
    'pc_gamer': 'pc_gamer',
    'portateis_cozinha_e_preparadores_de_alimentos': 'portable_kitchen_food_preparers',
    'unknown': 'unknown'
}

products['product_category_name_english'] = products['product_category_name_english'].fillna(
    products['product_category_name'].map(manual_map)
)

products[['product_category_name', 'product_category_name_english']].head()

KeyError: 'product_category_name_english'

New Feature: We calculate the physical volume and density. These features are essential for logistics and supply chain analysis. Volume helps determine warehouse space requirements, while density helps identify "bulky" items versus "compact" items.

In [ ]:
products['product_volume_cm3'] = (
    products['product_length_cm'] * products['product_height_cm'] * products['product_width_cm']
)

products['product_density_g_cm3'] = products['product_weight_g'] / (products['product_volume_cm3'] + 0.001)

products[['product_id', 'product_volume_cm3', 'product_density_g_cm3']].head()

,product_id,product_volume_cm3,product_density_g_cm3
0,1e9e8ef04dbcff4541ed26657ea517e5,2240.0,0.100446
1,3aa071139cb16b67ca9e5dea641aaa2f,10800.0,0.092593
2,96bd76ec8810374ed1b65e291975717f,2430.0,0.063374
3,cef67bcfe19066a932b7673e239eb23d,2704.0,0.137204
4,9dc1a7de274444849c219cff195d0b71,4420.0,0.141403


New Feature: The raw dataset contains over 70 unique product categories. By mapping these into 8-10 major Departments, we provide a structured hierarchy for the dashboard.

In [ ]:
department_map = {
    'bed_bath_table': 'Home_Living',
    'furniture_decor': 'Home_Living',
    'housewares': 'Home_Living',
    'furniture_living_room': 'Home_Living',
    'kitchen_dining_laundry_garden_furniture': 'Home_Living',
    'garden_tools': 'Home_Living',
    'home_confort': 'Home_Living',
    'home_appliances': 'Home_Living',
    'home_appliances_2': 'Home_Living',
    'furniture_bedroom': 'Home_Living',
    'furniture_mattress_and_upholstery': 'Home_Living',
    'flowers': 'Home_Living',
    'home_construction': 'Home_Living',
    'air_conditioning': 'Home_Living',
    'small_appliances_home_oven_and_coffee': 'Home_Living',
    'small_appliances': 'Home_Living',
    'home_comfort_2': 'Home_Living',

    'computers_accessories': 'Electronics_Tech',
    'telephony': 'Electronics_Tech',
    'computers': 'Electronics_Tech',
    'electronics': 'Electronics_Tech',
    'audio': 'Electronics_Tech',
    'fixed_telephony': 'Electronics_Tech',
    'tablets_printing_image': 'Electronics_Tech',
    'consoles_games': 'Electronics_Tech',
    'pc_gamer': 'Electronics_Tech',

    'health_beauty': 'Health_Beauty_Baby',
    'perfumery': 'Health_Beauty_Baby',
    'baby': 'Health_Beauty_Baby',
    'diapers_and_hygiene': 'Health_Beauty_Baby',

    'watches_gifts': 'Fashion_Accessories',
    'fashion_bags_accessories': 'Fashion_Accessories',
    'fashion_shoes': 'Fashion_Accessories',
    'fashion_underwear_beach': 'Fashion_Accessories',
    'fashion_male_clothing': 'Fashion_Accessories',
    'fashion_female_clothing': 'Fashion_Accessories',
    'fashion_childrens_clothes': 'Fashion_Accessories',
    'luggage_accessories': 'Fashion_Accessories',
    'fashion_sport': 'Fashion_Accessories',

    'sports_leisure': 'Sports_Hobbies_Leisure',
    'toys': 'Sports_Hobbies_Leisure',
    'musical_instruments': 'Sports_Hobbies_Leisure',
    'music': 'Sports_Hobbies_Leisure',
    'books_general_interest': 'Sports_Hobbies_Leisure',
    'books_technical': 'Sports_Hobbies_Leisure',
    'books_imported': 'Sports_Hobbies_Leisure',
    'art': 'Sports_Hobbies_Leisure',
    'arts_and_craftmanship': 'Sports_Hobbies_Leisure',
    'dvds_blu_ray': 'Sports_Hobbies_Leisure',
    'cds_dvds_musicals': 'Sports_Hobbies_Leisure',
    'cine_photo': 'Sports_Hobbies_Leisure',
    'cool_stuff': 'Sports_Hobbies_Leisure',
    'party_supplies': 'Sports_Hobbies_Leisure',
    'christmas_supplies': 'Sports_Hobbies_Leisure',

    'office_furniture': 'Business_Construction',
    'industry_commerce_and_business': 'Business_Construction',
    'stationery': 'Business_Construction',
    'agro_industry_and_commerce': 'Business_Construction',
    'construction_tools_construction': 'Business_Construction',
    'construction_tools_lights': 'Business_Construction',
    'construction_tools_safety': 'Business_Construction',
    'signaling_and_security': 'Business_Construction',
    'security_and_services': 'Business_Construction',
    'costumes_accessories': 'Business_Construction',

    'food': 'Food_Drink',
    'drinks': 'Food_Drink',
    'food_drink': 'Food_Drink',
    'la_cuisine': 'Food_Drink',

    'pet_shop': 'Other',
    'market_place': 'Other',
    'auto': 'Other'
}

products['product_department'] = products['product_category_name_english'].map(department_map).fillna('Other')

print("Feature Engineering Complete: 71 categories mapped.")
products['product_department'].value_counts()

Feature Engineering Complete: 71 categories mapped.


product_department
Home_Living               10280
Sports_Hobbies_Leisure     5995
Health_Beauty_Baby         4243
Electronics_Tech           3823
Other                      3497
Fashion_Accessories        2872
Business_Construction      1964
Food_Drink                  277
Name: count, dtype: int64

After successfully engineering the product_volume_cm3 and product_density_g_cm3 features, the raw dimension columns (length, height, width) are no longer required for high-level analysis. Removing them optimizes the dataset size and simplifies the schema for the upcoming SQL integration and visualization.

In [ ]:
cols_to_drop = [ 
    'product_category_name_english_x',
    'product_category_name_english_y'
]
products = products.drop(columns=cols_to_drop)
products.head()

,product_id,product_category_name,product_description_lenght,product_photos_qty,product_weight_g,product_category_name_english,product_volume_cm3,product_density_g_cm3,product_department
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,287.0,1.0,225.0,perfumery,2240.0,0.100446,Health_Beauty_Baby
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,276.0,1.0,1000.0,art,10800.0,0.092593,Sports_Hobbies_Leisure
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,250.0,1.0,154.0,sports_leisure,2430.0,0.063374,Sports_Hobbies_Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,261.0,1.0,371.0,baby,2704.0,0.137204,Health_Beauty_Baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,402.0,4.0,625.0,housewares,4420.0,0.141403,Home_Living


Sellers Geographic Standardization:
We clean the sellers' location data by removing leading/trailing spaces and standardizing the casing. This ensures that geographic analysis in Power BI correctly groups sellers by city and state without duplicates.

In [ ]:
sellers['seller_city'] = sellers['seller_city'].str.strip().str.title()
sellers['seller_state'] = sellers['seller_state'].str.strip().str.upper()

print(f"Unique States: {sellers['seller_state'].unique()}")
sellers.head()

Unique States: <StringArray>
['SP', 'RJ', 'PE', 'PR', 'GO', 'SC', 'BA', 'DF', 'RS', 'MG', 'RN', 'MT', 'CE',
 'PB', 'AC', 'ES', 'RO', 'PI', 'MS', 'SE', 'MA', 'AM', 'PA']
Length: 23, dtype: str


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,Campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,Mogi Guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,Rio De Janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,Sao Paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,Braganca Paulista,SP


Before exporting, we perform a final check to ensure that primary keys are unique and non-null. This is a critical step to guarantee referential integrity before loading the data into SQL.

In [ ]:
assert products['product_id'].is_unique
assert sellers['seller_id'].is_unique
assert products['product_id'].notnull().all()

print("✅ Integrity Check Passed: No duplicates or null IDs.")

✅ Integrity Check Passed: No duplicates or null IDs.


We save the fully cleaned and optimized datasets into the processed folder.

In [44]:
products.to_csv('../data/processed/products_cleaned.csv', index=False)
sellers.to_csv('../data/processed/sellers_cleaned.csv', index=False)

print("🚀 Success! All files cleaned and saved.")

🚀 Success! All files cleaned and saved.
